In [33]:
%load_ext sql
%sql sqlite:///pokemon.sqlite
from utils import *

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [40]:
def view_trainadores(limite=None, mostrar=False):
    treinadores = Trainer.objects.all()
    if limite:
        treinadores = treinadores[:limite]
        
    for treinador in treinadores:
        pokemons = treinador.pokemons.all()
        treinador.avg_level = (
            sum([p.pokelevel for p in pokemons]) 
            / len(pokemons)
        ) if pokemons else 0
        
    render('trainers.html', context={'trainers': treinadores}, show=mostrar)

In [35]:
# Visualizando o que a view retorna

view_trainadores(limite=3, mostrar=True)

In [36]:
Trainer.objects.count()

11860

In [ ]:
timeit(view_trainadores)

# Investigando a quantidade de consultas ao Banco de Dados

In [41]:
with count_queries():
    view_trainadores(limite=100)

Number of queries: 528


In [ ]:
display_template('trainers.html')

In [ ]:
with count_queries() as m:
    treinadores = Trainer.objects.all()[:100]
    
    for t in treinadores:
        t.favorite_pokemon.name
        
pretty(m.queries[:10])

In [ ]:
consulta_treinadores = m.queries[0]
print(consulta_treinadores)

In [ ]:
%sql $consulta_treinadores

In [ ]:
consulta_pokemon = m.queries[1]
print(consulta_pokemon)

In [ ]:
%sql $consulta_pokemon

In [ ]:
query = (
    'SELECT * '
    'FROM "Trainer" LEFT OUTER JOIN "PokemonKind"'
    'ON ("Trainer"."favorite_pokemon_id" = "PokemonKind"."id")'
    'LIMIT 100'
)
%sql $query

In [ ]:
with count_queries() as m:
    treinadores = Trainer.objects.select_related('favorite_pokemon').all()[:100]
    
    for t in treinadores:
        t.favorite_pokemon.name
        
print(m.queries[0])

In [38]:
def view_trainadores(limite=None):
    treinadores = Trainer.objects.select_related("favorite_pokemon").all()
    
    if limite:
        treinadores = treinadores[:limite]
        
    for treinador in treinadores:
        pokemons = treinador.pokemons.all()
        treinador.avg_level = (
            sum([p.pokelevel for p in pokemons]) 
            / len(pokemons)
        ) if pokemons else 0
        
    render('trainers.html', context={'trainers': treinadores})

In [39]:
with count_queries():
    view_trainadores(limite=100)

Number of queries: 428
